# RAS Commander: Sequential Plan Execution

This notebook demonstrates how to sequentially execute multiple HEC-RAS plans using the RAS Commander library. Sequential execution is useful for batch processing plans that need to be run in a specific order or when you want to ensure consistent resource usage across multiple runs.

## Operations Covered

1. **Project Initialization**: Initialize a HEC-RAS project by specifying the project path and version
2. **Sequential Execution of All Plans**: Run all plans in a project sequentially in a test folder
3. **Selective Plan Execution**: Run only specific plans in sequence
4. **Geometry Preprocessor Management**: Clear geometry preprocessor files before execution
5. **Execution Result Analysis**: Track and analyze the results of sequential executions
6. **Performance Monitoring**: Monitor and compare execution times across different runs

Let's begin by importing the necessary libraries and setting up our environment.

## Package Installation and Environment Setup
Uncomment and run package installation commands if needed

In [1]:
# Install ras-commander from pip (uncomment to install if needed)
#!pip install ras-commander
# This installs ras-commander and all dependencies

In [ ]:
# Import all required modules

# Import all ras-commander modules
from ras_commander import *

# Import the required libraries for this notebook
import numpy as np
import pandas as pd
from IPython import display
from pathlib import Path
import matplotlib.pyplot as plt
import psutil  # For getting system CPU info
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import subprocess
import sys
import os
import shutil

In [ ]:
# Extract the Bald Eagle Creek example project
# The extract_project method downloads the project from GitHub if not already present,
# and extracts it to the example_projects folder
bald_eagle_path = RasExamples.extract_project("Balde Eagle Creek")
print(f"Extracted project to: {bald_eagle_path}")  


# Verify the path exists
print(f"Bald Eagle Creek project exists: {bald_eagle_path.exists()}")

In [ ]:
# define examples_dir as parent of bald_eagle_path
examples_dir = bald_eagle_path.parent
print(f"Examples directory set to: {examples_dir}")

    
# Remove any compute test folders from previous runs
for folder in examples_dir.glob("*[[]AllSequential[]]*"):
    if folder.is_dir():
        print(f"Removing existing test folder: {folder}")
        shutil.rmtree(folder)
        
for folder in examples_dir.glob("*[[]SpecificSequential*[]]*"):
    if folder.is_dir():
        print(f"Removing existing test folder: {folder}")
        shutil.rmtree(folder)


## Understanding Sequential Execution in HEC-RAS

HEC-RAS simulations can be executed in several ways:

1. **Single Plan Execution**: Run one plan at a time using `RasCmdr.compute_plan()`
2. **Sequential Execution**: Run multiple plans one after another using `RasCmdr.compute_test_mode()`
3. **Parallel Execution**: Run multiple plans simultaneously using `RasCmdr.compute_parallel()`

This notebook focuses on the second approach: **Sequential Execution**. Here are the key benefits of sequential execution:

- **Controlled Resource Usage**: By running plans one at a time, you ensure consistent resource usage
- **Dependency Management**: When later plans depend on results from earlier plans
- **Simplified Debugging**: Easier to identify which plan is causing an issue when they run sequentially
- **Consistent Test Environment**: All plans run in the same isolated folder

The `compute_test_mode()` function from `RasCmdr` is specifically designed for this purpose. It creates a separate test folder, copies the project there, and executes the specified plans in sequential order.

## Downloading and Extracting Example HEC-RAS Project

Let's use the `RasExamples` class to download and extract the "Balde Eagle Creek" example project.

## Step 1: Project Initialization

Let's initialize the HEC-RAS project using the `init_ras_project()` function.

In [ ]:
# Initialize the HEC-RAS project
init_ras_project(bald_eagle_path, "6.6")
print(f"Initialized HEC-RAS project: {ras.project_name}")

# Display the current plan files in the project
print("\nHEC-RAS Project Plan Data:")
display.display(ras.plan_df)

# Check how many plans we have
plan_count = len(ras.plan_df)
print(f"Found {plan_count} plans in the project")

## Understanding the RasCmdr.compute_test_mode Method

Before we start executing plans, let's understand the `compute_test_mode()` method from the `RasCmdr` class, which we'll use for sequential execution.

### Key Parameters

- `plan_number` (str, list[str], optional): Plan number or list of plan numbers to execute. If None, all plans will be executed.
- `dest_folder_suffix` (str, optional): Suffix to append to the test folder name. Defaults to "[Test]".
- `clear_geompre` (bool, optional): Whether to clear geometry preprocessor files. Defaults to False.
- `num_cores` (int, optional): Maximum number of cores to use for each plan. If None, the current setting is not changed.
- `ras_object` (RasPrj, optional): Specific RAS object to use. If None, uses the global ras object.
- `overwrite_dest` (bool, optional): Whether to overwrite the destination folder if it exists. Defaults to False.

### Return Value
- `Dict[str, bool]`: Dictionary of plan numbers and their execution success status.

### Key Concepts

1. **Test Folder**: The function creates a separate folder with the specified suffix, copying the project there for execution.
2. **Sequential Execution**: Plans are executed one after another in the specified order.
3. **Geometry Preprocessor Files**: These files store precomputed hydraulic properties. Clearing them forces HEC-RAS to recompute these properties.
4. **Destination Folder Option**: The suffix determines the name of the test folder. Unlike `compute_plan()`, you can't specify an arbitrary destination folder.
5. **Overwrite Option**: Controls whether an existing test folder should be overwritten.

Now, let's see how this works in practice.

## Step 2: Sequential Execution of All Plans

Let's execute all plans in the project sequentially. This will create a test folder with the suffix "[AllSequential]" and run all plans one after another.

In [ ]:
print("Executing all plans sequentially...")
print("This may take several minutes...")

# Record start time for performance measurement
start_time = time.time()

# Execute all plans sequentially
# - dest_folder_suffix: Suffix to append to the test folder name
# - overwrite_dest: Overwrite the destination folder if it exists
# - no ras object is specified, it will use the default "ras" object
execution_results = RasCmdr.compute_test_mode(
    dest_folder_suffix="[AllSequential]",
    overwrite_dest=True
)

# Record end time and calculate duration
end_time = time.time()
total_duration = end_time - start_time

print(f"Sequential execution of all plans completed in {total_duration:.2f} seconds")

# Create a DataFrame from the execution results for better visualization
results_df = pd.DataFrame([
    {"Plan": plan, "Success": success}
    for plan, success in execution_results.items()
])

# Display the results
print("\nExecution Results:")
display.display(results_df)

## Step 3: Examining the Test Folder

Let's examine the test folder created by `compute_test_mode()` to better understand what happened during sequential execution.

In [ ]:
# Define the test folder path
test_folder = script_dir / f"{ras.project_name} [AllSequential]"

if test_folder.exists():
    print(f"Test folder exists: {test_folder}")
    
    # List the key files in the test folder
    print("\nKey files in test folder:")
    
    # First, list the project file and all plan files
    prj_files = list(test_folder.glob("*.prj"))
    plan_files = list(test_folder.glob("*.p*"))
    plan_files.sort()
    
    if prj_files:
        print(f"Project file: {prj_files[0].name}")
    
    print("Plan files:")
    for plan_file in plan_files:
        file_size = plan_file.stat().st_size / 1024  # Size in KB
        print(f"  {plan_file.name}: {file_size:.1f} KB")
    
    # Look for HDF result files
    hdf_files = list(test_folder.glob("*.hdf"))
    hdf_files.sort()
    
    print("\nHDF files:")
    for hdf_file in hdf_files:
        file_size = hdf_file.stat().st_size / (1024 * 1024)  # Size in MB
        print(f"  {hdf_file.name}: {file_size:.1f} MB")
    
    # Geometry preprocessor files (if any)
    geompre_files = list(test_folder.glob("*.c*"))
    geompre_files.sort()
    
    if geompre_files:
        print("\nGeometry preprocessor files:")
        for geompre_file in geompre_files:
            file_size = geompre_file.stat().st_size / 1024  # Size in KB
            print(f"  {geompre_file.name}: {file_size:.1f} KB")
    else:
        print("\nNo geometry preprocessor files found")
        
    # Initialize a RAS project in the test folder to inspect results
    try:
        test_ras = RasPrj()
        init_ras_project(test_folder, ras.ras_exe_path, ras_object=test_ras)
        print("\nPlans with results in the test folder:")
        test_plans_with_results = test_ras.plan_df[test_ras.plan_df['HDF_Results_Path'].notna()]
        display.display(test_plans_with_results[['plan_number', 'HDF_Results_Path']])
    except Exception as e:
        print(f"Error initializing test folder as a RAS project: {e}")
else:
    print(f"Test folder not found: {test_folder}")

## Step 4: Sequential Execution of Specific Plans

Now, let's execute only specific plans in the project. We'll select plans "01" and "02" and run them sequentially with the `clear_geompre` option set to True.

In [ ]:
print("Executing specific plans sequentially with clearing geometry preprocessor files...")
print("This may take several minutes...")

# Define the plans to execute
selected_plans = ["01", "02"]
print(f"Selected plans: {', '.join(selected_plans)}")

# Record start time for performance measurement
start_time = time.time()

# Execute specific plans sequentially
# - plan_number: List of plan numbers to execute
# - dest_folder_suffix: Suffix to append to the test folder name
# - clear_geompre: Clear geometry preprocessor files before execution
# - overwrite_dest: Overwrite the destination folder if it exists
execution_results = RasCmdr.compute_test_mode(
    plan_number=selected_plans,
    dest_folder_suffix="[SpecificSequentialClearGeompre]",
    clear_geompre=True,
    overwrite_dest=True
)

# Record end time and calculate duration
end_time = time.time()
total_duration = end_time - start_time

print(f"Sequential execution of specific plans completed in {total_duration:.2f} seconds")

# Create a DataFrame from the execution results for better visualization
results_df = pd.DataFrame([
    {"Plan": plan, "Success": success}
    for plan, success in execution_results.items()
])

# Display the results
print("\nExecution Results:")
display.display(results_df)

## Summary of Sequential Plan Execution

In this notebook, we've explored how to execute HEC-RAS plans sequentially using the RAS Commander library. Here's a summary of the key techniques we've covered:

1. **Project Setup and Initialization**: Setting up the environment and initializing a HEC-RAS project
2. **Example Project Management**: Using `RasExamples` to download and extract sample projects
3. **Basic Sequential Execution**: Using `RasCmdr.compute_test_mode()` to run all plans in a project
4. **Test Folder Analysis**: Examining the contents and results of sequential execution
5. **Selective Plan Execution**: Running specific plans with geometry preprocessor clearing

### Key Functions Used

- `init_ras_project()`: Initialize a HEC-RAS project
- `RasExamples.extract_project()`: Extract example projects for testing
- `RasCmdr.compute_test_mode()`: Run plans sequentially in a test folder
- `Path.glob()`: Examine test folder contents and results
- `RasCmdr.compute_test_mode(clear_geompre=True)`: Execute plans with preprocessor clearing

### Best Practices for Sequential Execution

1. **Environment Setup**: Ensure all required libraries are installed and properly imported
2. **Project Organization**: Clean up existing test folders before new executions
3. **Resource Management**: Monitor system resources (CPU cores, memory) for optimal performance
4. **Test Folder Naming**: Use meaningful suffixes to distinguish different execution runs
5. **Performance Tracking**: Monitor execution times for each sequential run
6. **Results Verification**: Check test folders for successful plan execution and result files
7. **Selective Execution**: Use plan filtering when only specific plans need to be run

With these techniques, you can effectively manage and execute HEC-RAS simulations sequentially, whether running all plans or a selected subset with specific configurations.